In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

In [3]:
all_x, all_y, points_number = get_train_data()

In [4]:
def get_resized_train_data(train_x, train_y, shape=(64, 64, 3)):
    resized_x = np.array([resize(image, shape, mode='reflect') for image in train_x])
    resized_y = np.array([[
            int(coordinate * shape[i % 2] / np.shape(image)[i % 2])
            for i, coordinate in enumerate(answer)
        ] for image, answer in zip(train_x, train_y)
    ])
    return resized_x, resized_y

In [5]:
resized_x, resized_y = get_resized_train_data(all_x, all_y)
input_shape=np.shape(resized_x[0])

In [6]:
train_x, test_x, train_y, test_y = train_test_split(
    resized_x, resized_y, test_size=0.3, random_state=42
)

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [9]:
datagen.fit(train_x)

In [10]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

model.add(Dense(points_number * 2, activation='relu'))

In [11]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['mse'])

In [ ]:
model.fit_generator(datagen.flow(train_x, train_y, batch_size=32),
                    steps_per_epoch=int(len(train_y) / 32), epochs=16)

Epoch 1/16
131/131 [==============================] - 34s - loss: 486.3416 - mean_squared_error: 486.3416    
Epoch 2/16
131/131 [==============================] - 31s - loss: 351.1515 - mean_squared_error: 351.1515    
Epoch 3/16
131/131 [==============================] - 31s - loss: 350.8481 - mean_squared_error: 350.8481    
Epoch 4/16
131/131 [==============================] - 31s - loss: 350.5595 - mean_squared_error: 350.5595    
Epoch 5/16
131/131 [==============================] - 31s - loss: 350.8810 - mean_squared_error: 350.8810    
Epoch 6/16
131/131 [==============================] - 31s - loss: 350.7498 - mean_squared_error: 350.7498    
Epoch 7/16
131/131 [==============================] - 31s - loss: 350.7006 - mean_squared_error: 350.7006    
Epoch 8/16
131/131 [==============================] - 31s - loss: 350.6621 - mean_squared_error: 350.6621    
Epoch 9/16
131/131 [==============================] - 31s - loss: 350.9129 - mean_squared_error: 350.9129    
Epoch 10/1